# fastNLP 1分钟上手教程

## step 1
读取数据集

In [1]:
import sys
sys.path.append("../")

from fastNLP import DataSet

data_path = "./sample_data/tutorial_sample_dataset.csv"
ds = DataSet.read_csv(data_path, headers=('raw_sentence', 'label'), sep='\t')

c:\users\zyfeng\miniconda3\envs\fastnlp\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
ds[1]

{'raw_sentence': This quiet , introspective and entertaining independent is worth seeking . type=str,
'label': 4 type=str}

## step 2
数据预处理
1. 类型转换
2. 切分验证集
3. 构建词典

In [3]:
# 将所有数字转为小写
ds.apply(lambda x: x['raw_sentence'].lower(), new_field_name='raw_sentence')
# label转int
ds.apply(lambda x: int(x['label']), new_field_name='label_seq', is_target=True)

def split_sent(ins):
    return ins['raw_sentence'].split()
ds.apply(split_sent, new_field_name='words', is_input=True)


In [4]:
# 分割训练集/验证集
train_data, dev_data = ds.split(0.3)
print("Train size: ", len(train_data))
print("Test size: ", len(dev_data))

Train size:  54
Test size:  23


In [5]:
from fastNLP import Vocabulary
vocab = Vocabulary(min_freq=2)
train_data.apply(lambda x: [vocab.add(word) for word in x['words']])

# index句子, Vocabulary.to_index(word)
train_data.apply(lambda x: [vocab.to_index(word) for word in x['words']], new_field_name='word_seq', is_input=True)
dev_data.apply(lambda x: [vocab.to_index(word) for word in x['words']], new_field_name='word_seq', is_input=True)


## step 3
 定义模型

In [6]:
from fastNLP.models import CNNText
model = CNNText(embed_num=len(vocab), embed_dim=50, num_classes=5, padding=2, dropout=0.1)


## step 4
开始训练

In [7]:
from fastNLP import Trainer, CrossEntropyLoss, AccuracyMetric
trainer = Trainer(model=model, 
                  train_data=train_data, 
                  dev_data=dev_data,
                  loss=CrossEntropyLoss(),
                  metrics=AccuracyMetric()
                  )
trainer.train()
print('Train finished!')


input fields after batch(if batch size is 2):
	words: (1)type:numpy.ndarray (2)dtype:object, (3)shape:(2,) 
	word_seq: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 25]) 
target fields after batch(if batch size is 2):
	label_seq: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2019-01-12 17-00-48


Evaluation at Epoch 1/3. Step:2/6. AccuracyMetric: acc=0.173913
Evaluation at Epoch 2/3. Step:4/6. AccuracyMetric: acc=0.26087
Evaluation at Epoch 3/3. Step:6/6. AccuracyMetric: acc=0.304348

In Epoch:3/Step:6, got best dev performance:AccuracyMetric: acc=0.304348
Reloaded the best model.
Train finished!


### 本教程结束。更多操作请参考进阶教程。